# Terrain Classification - Preprocessed

#### This notebook runs classification on preprocessed .csv data which has been integrated from all three modules

## Part 1 - Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import sklearn
import pymrmr

In [12]:
# Import already processed features
vectors = {}

for filename in glob.glob('processed_data/integrated/*.csv'):
    vector_name = filename.split('.')[0].split('/')[-1]
    vectors.update({vector_name: pd.read_csv(filename)})

print('Vector Keys: {}'.format(list(vectors.keys())))

Vector Keys: ['Features', 'PSDLogs', 'FFTs']


In [14]:
vectors['Features'].tail()

,Label,Autocorrelation Z Accel Left,Excess Kurtosis Z Accel Left,L2 Norm Z Accel Left,Max Z Accel Left,Mean Z Accel Left,Min Z Accel Left,Root Mean Squared Z Accel Left,Skew Z Accel Left,Std Dev Z Accel Left,...,Min Y Accel Left,Root Mean Squared Y Accel Left,Skew Y Accel Left,Std Dev Y Accel Left,Zero Crossing Rate Y Accel Left,Frequency Center Y Accel Left,Mean Square Frequency Y Accel Left,Root Mean Square Frequency Y Accel Left,Root Variance Frequency Y Accel Left,Variance Frequency Y Accel Left
1377,5,2.141245,0.392100,1.927656,-0.011501,-1.904700,-2.396258,1.927656,-1.257591,1.306408,...,-1.058982,0.584913,1.449927,-0.252706,-0.222700,-0.130463,-0.389301,-0.184960,-0.184960,0.145356
1378,5,-0.777689,-0.737330,-0.804340,-1.158096,-0.754491,0.882625,-0.804340,0.310546,-1.007358,...,-0.161647,-0.742767,-0.623699,1.182530,0.858984,-0.359586,0.791737,0.872964,0.872964,0.175807
1379,5,-0.657774,-0.568196,-0.630261,-0.403790,0.120612,0.685748,-0.630261,0.041877,-0.319306,...,1.399793,1.078151,-1.647667,-0.624513,-0.763542,-0.321868,-0.926275,-0.926600,-0.926600,0.169425
1380,5,0.845368,0.715389,0.952692,0.279878,0.111183,-2.068473,0.952692,-1.159586,1.293368,...,-0.851668,-1.162916,0.075878,0.152159,-0.222700,-0.173144,0.832737,0.903253,0.903253,0.153502
1381,5,-0.013884,0.369146,0.145673,0.799567,0.135607,-0.700639,0.145673,0.301371,0.474124,...,-1.109988,1.196439,0.073766,-1.537922,-0.763542,8.869341,0.099085,0.306889,0.306889,-11.262714


In [38]:
# Get Z Gyro columns
z_gyro_columns = [column for column in vectors['Features'].columns if 'Z Gyro' in column]
z_gyro_columns.insert(0, 'Label')
        
vectors['Features'][z_gyro_columns].tail()

,Label,Autocorrelation Z Gyro Left,Excess Kurtosis Z Gyro Left,L2 Norm Z Gyro Left,Max Z Gyro Left,Mean Z Gyro Left,Min Z Gyro Left,Root Mean Squared Z Gyro Left,Skew Z Gyro Left,Std Dev Z Gyro Left,Zero Crossing Rate Z Gyro Left,Frequency Center Z Gyro Left,Mean Square Frequency Z Gyro Left,Root Mean Square Frequency Z Gyro Left,Root Variance Frequency Z Gyro Left,Variance Frequency Z Gyro Left
1377,5,1.462255,-1.103557,1.264565,1.508585,1.177764,1.192112,1.264565,0.266957,1.035429,-0.304212,-0.587289,-0.600225,-0.658042,-0.658042,0.445186
1378,5,0.023425,-0.973332,0.190493,-0.188312,0.255453,0.576537,0.190493,-0.713850,-0.537835,-0.304212,-0.575625,-0.657671,-0.822287,-0.822287,0.442216
1379,5,0.100980,-0.003507,0.257840,-0.263159,0.342276,0.670416,0.257840,-1.159659,-1.076589,-0.304212,0.538733,-0.204828,0.049236,0.049236,-0.115116
1380,5,-0.741767,2.752968,-0.578212,-0.549822,-0.444793,-0.367506,-0.578212,0.070403,-0.963735,-0.304212,3.998491,2.852942,2.409326,2.409326,-5.396679
1381,5,-1.594241,2.530297,-2.060976,-1.954664,-1.821564,-1.041728,-2.060976,1.990739,-1.590260,-0.304212,-0.090913,-0.681485,-0.904221,-0.904221,0.268017


## Part 2 - Minimum Redundancy Maximum Relevance (mRMR) Feature Selection

### Part (a) - Compare feature selection methods

*MID* - Mutual Information Difference

*MIQ* - Mutual Information Quotient

In [17]:
pymrmr.mRMR(data=vectors['Features'], method='MID', nfeats=20)

['Zero Crossing Rate X Accel Left',
 'Zero Crossing Rate Y Accel Left',
 'Zero Crossing Rate X Gyro Left',
 'Excess Kurtosis Y Accel Left',
 'Frequency Center Z Gyro Left',
 'Frequency Center Y Accel Left',
 'Mean Square Frequency Z Accel Left',
 'Zero Crossing Rate Y Gyro Left',
 'Mean Square Frequency X Accel Left',
 'Excess Kurtosis Y Gyro Left',
 'Min Y Accel Left',
 'Zero Crossing Rate Z Gyro Left',
 'Excess Kurtosis X Accel Left',
 'Min X Accel Left',
 'Frequency Center Y Gyro Left',
 'Zero Crossing Rate Z Accel Left',
 'Max Y Accel Left',
 'Mean Square Frequency Y Accel Left',
 'Excess Kurtosis Z Gyro Left',
 'Frequency Center X Accel Left']

In [16]:
pymrmr.mRMR(data=vectors['Features'], method='MIQ', nfeats=20)

['Zero Crossing Rate X Accel Left',
 'Frequency Center Z Gyro Left',
 'Excess Kurtosis Y Accel Left',
 'Zero Crossing Rate X Gyro Left',
 'Zero Crossing Rate Y Accel Left',
 'Zero Crossing Rate Y Gyro Left',
 'Zero Crossing Rate Z Gyro Left',
 'Mean Square Frequency Z Accel Left',
 'Excess Kurtosis Y Gyro Left',
 'Excess Kurtosis X Accel Left',
 'Mean Square Frequency X Accel Left',
 'Min Y Accel Left',
 'Frequency Center Y Accel Left',
 'Min X Accel Left',
 'Frequency Center Y Gyro Left',
 'Zero Crossing Rate Z Accel Left',
 'Excess Kurtosis Z Gyro Left',
 'Max Y Accel Left',
 'Mean Square Frequency Y Accel Left',
 'Variance Frequency X Accel Left']

### Part (b) - Compare Feature Vectors

In [ ]:
pymrmr.mRMR(data=vectors['Features'], method='MID', nfeats=10)

In [ ]:
pymrmr.mRMR(data=vectors['FFTs'], method='MID', nfeats=10)

In [ ]:
pymrmr.mRMR(data=vectors['PSDLogs'], method='MID', nfeats=10)